# Assess RNA binding re-parameterisation

The RNA binding data from paper ["Metabolic engineering of Escherichia coli using synthetic small regulatory RNAs"](https://www.nature.com/articles/nbt.2461#Fig1) 

In [1]:
from copy import deepcopy
from functools import partial
import os
import sys

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('bmh')
plt.style.use('seaborn-v0_8')
import numpy as np
import jax.numpy as jnp
import jax.random as jr
import pandas as pd
import jax
import diffrax as dfx
from tqdm import tqdm

# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.8"
os.environ["TF_CPP_MIN_LOG_LOVEL"] = "0"
jax.config.update('jax_platform_name', 'gpu')

import bioreaction
from bioreaction.model.data_containers import Impulse
from bioreaction.simulation.basic_sim import basic_de_sim, convert_model, BasicSimParams, BasicSimState, bioreaction_sim_expanded
from bioreaction.model.data_tools import construct_model_fromnames
from bioreaction.simulation.med_sim import get_int_med_model, basic_de, MedSimParams, MedSimInternalState


if __package__ is None:

    module_path = os.path.abspath(os.path.join('..'))
    sys.path.append(module_path)

    __package__ = os.path.basename(module_path)

from tests.shared import five_circuits, CONFIG
import src
from src.srv.sequence_exploration.sequence_analysis import b_tabulate_mutation_info
from src.utils.misc.numerical import make_symmetrical_matrix_from_sequence, triangular_sequence
from src.utils.misc.type_handling import flatten_listlike
from src.utils.misc.units import per_mol_to_per_molecule
from src.utils.modelling.deterministic import bioreaction_sim_dfx_expanded
from src.utils.results.analytics.naming import get_true_names_analytics, get_true_interaction_cols
from src.utils.results.analytics.timeseries import generate_analytics, get_precision, get_sensitivity


config = deepcopy(CONFIG)


Examining previously generated RNA sequences for which the $\Delta G$ has been simulated and translated into 

In [7]:
path_to_stats = 'data/ensemble_generate_circuits/2023_02_24_170946/gather_interaction_stats/circuit_stats.csv'
path_to_stats = os.path.join('..', path_to_stats)

circuit_stats = pd.read_csv(path_to_stats)

In [ ]:
energy_cols = get_true_interaction_cols(circuit_stats, 'energies')

sns.scatterplot(pd.concat(circuit_stats[get_true_interaction_cols]), )